In [3]:
"""
Integrated Driver Security and Drowsiness Detection System

This system combines:
1. Face recognition security verification (runs first)
2. Driver drowsiness and yawning detection (runs after security verification)

Requirements:
1. Install packages: pip install opencv-python dlib numpy scipy pygame face-recognition
2. Download shape_predictor_68_face_landmarks.dat from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
3. Create 'Images_sec' folder with authorized driver photos (e.g., Ahmed.jpg, Bill.jpeg, etc.)
4. Place your MP3 alert file "Untitled (1).mp3" in the same directory
5. Ensure webcam is connected and working

Usage:
- Run the script and position your face clearly in front of the camera
- First, the security system will verify your identity (needs 2 consecutive verifications)
- After successful verification, drowsiness monitoring begins automatically
- The system will play "Untitled (1).mp3" when drowsiness or yawning is detected
- Press 'q' to quit at any time
"""

import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import pygame
import time
import sys
import face_recognition
import os
import datetime

class IntegratedDriverSafetySystem:
    def __init__(self, security_images_path="Images_sec", mp3_filename="Untitled (1).mp3"):
        # Security system initialization
        self.security_images_path = security_images_path
        self.authorized_drivers = []
        self.driver_names = []
        self.load_authorized_drivers()
        
        # Drowsiness detection initialization
        self.detector = None
        self.predictor = None
        self.init_drowsiness_detector()
        
        # System state
        self.security_verified = False
        self.verified_driver_name = ""
        self.consecutive_verifications = 0
        self.last_verified_name = ""
        
        # Audio setup
        self.mp3_file = mp3_filename
        self.setup_audio()
        
        # Drowsiness detection parameters
        self.LEFT_EYE = [36, 37, 38, 39, 40, 41]
        self.RIGHT_EYE = [42, 43, 44, 45, 46, 47]
        self.MOUTH = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
        
        # Thresholds
        self.EYE_AR_THRESH = 0.25
        self.MOUTH_AR_THRESH = 0.7
        self.EYE_AR_CONSEC_FRAMES = 20
        self.YAWN_CONSEC_FRAMES = 15
        
        # Counters for drowsiness detection
        self.eye_counter = 0
        self.yawn_counter = 0
        self.drowsy_alert = False
        self.yawn_alert = False
    
    def load_authorized_drivers(self):
        """Load and encode authorized driver faces"""
        try:
            driver_list = os.listdir(self.security_images_path)
            print(f"Found driver images: {driver_list}")
            
            images = []
            for driver_file in driver_list:
                img_path = os.path.join(self.security_images_path, driver_file)
                current_img = cv2.imread(img_path)
                if current_img is not None:
                    images.append(current_img)
                    self.driver_names.append(os.path.splitext(driver_file)[0])
            
            print(f"Authorized drivers: {self.driver_names}")
            
            # Encode faces
            for img in images:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                encodings = face_recognition.face_encodings(img_rgb)
                if encodings:
                    self.authorized_drivers.append(encodings[0])
                else:
                    print("Warning: Could not encode face in one of the images")
            
            print(f"Successfully encoded {len(self.authorized_drivers)} driver faces")
            
        except Exception as e:
            print(f"Error loading authorized drivers: {e}")
            print("Make sure 'Images_sec' folder exists with driver photos")
    
    def init_drowsiness_detector(self):
        """Initialize dlib components for drowsiness detection"""
        try:
            self.detector = dlib.get_frontal_face_detector()
            self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
            print("Drowsiness detection system initialized")
        except Exception as e:
            print(f"Error initializing drowsiness detector: {e}")
            print("Make sure 'shape_predictor_68_face_landmarks.dat' is available")
    
    def setup_audio(self):
        """Setup audio system for alerts"""
        try:
            pygame.mixer.init()
            self.alert_sound = pygame.mixer.Sound(self.mp3_file)
            print(f"Audio system ready - Using: {self.mp3_file}")
        except Exception as e:
            print(f"Audio setup failed: {e}")
            print("Will use system beep as fallback")
            self.alert_sound = None
    
    def play_alert(self):
        """Play alert sound"""
        if self.alert_sound:
            self.alert_sound.play()
        else:
            print('\a')  # System beep
            sys.stdout.flush()
    
    def security_verification(self, frame):
        """Handle security verification phase"""
        # Resize for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        
        # Find faces and encodings
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        current_frame_verified = False
        current_verified_name = ""
        
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # Compare with authorized drivers
            matches = face_recognition.compare_faces(self.authorized_drivers, face_encoding, tolerance=0.6)
            face_distances = face_recognition.face_distance(self.authorized_drivers, face_encoding)
            
            if len(face_distances) > 0:
                best_match_index = np.argmin(face_distances)
                
                if matches[best_match_index]:
                    name = self.driver_names[best_match_index].upper()
                    current_frame_verified = True
                    current_verified_name = name
                    
                    # Check consecutive verifications
                    if name == self.last_verified_name:
                        self.consecutive_verifications += 1
                    else:
                        self.consecutive_verifications = 1
                        self.last_verified_name = name
                    
                    # Draw verification info
                    top, right, bottom, left = face_location
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, f"{name} VERIFIED", (left + 6, bottom - 6), 
                              cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
                    
                    # Check if verification complete
                    if self.consecutive_verifications >= 2:
                        self.security_verified = True
                        self.verified_driver_name = name
                        return True
        
        # Reset if no verification in current frame
        if not current_frame_verified and self.consecutive_verifications > 0:
            self.consecutive_verifications = 0
            self.last_verified_name = ""
        
        # Display verification progress
        if self.consecutive_verifications > 0:
            cv2.putText(frame, f"Verifications: {self.consecutive_verifications}/2", 
                       (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        
        # Display instructions
        cv2.putText(frame, "SECURITY VERIFICATION", (50, 100), 
                   cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, "Look directly at camera", (50, 130), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        return False
    
    def eye_aspect_ratio(self, eye):
        """Calculate eye aspect ratio"""
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear
    
    def mouth_aspect_ratio(self, mouth):
        """Calculate mouth aspect ratio"""
        A = dist.euclidean(mouth[2], mouth[10])
        B = dist.euclidean(mouth[4], mouth[8])
        C = dist.euclidean(mouth[0], mouth[6])
        mar = (A + B) / (2.0 * C)
        return mar
    
    def extract_landmarks(self, gray, rect):
        """Extract facial landmarks"""
        shape = self.predictor(gray, rect)
        coords = np.zeros((68, 2), dtype="int")
        
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)
        
        return coords
    
    def drowsiness_monitoring(self, frame):
        """Handle drowsiness monitoring phase"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray, 0)
        
        for face in faces:
            landmarks = self.extract_landmarks(gray, face)
            
            # Extract eye and mouth coordinates
            left_eye = landmarks[self.LEFT_EYE]
            right_eye = landmarks[self.RIGHT_EYE]
            mouth = landmarks[self.MOUTH]
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = self.mouth_aspect_ratio(mouth)
            
            # Draw contours
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)
            
            # Drowsiness detection
            if ear < self.EYE_AR_THRESH:
                self.eye_counter += 1
                if self.eye_counter >= self.EYE_AR_CONSEC_FRAMES:
                    if not self.drowsy_alert:
                        self.drowsy_alert = True
                        self.play_alert()
            else:
                self.eye_counter = 0
                self.drowsy_alert = False
            
            # Yawning detection
            if mar > self.MOUTH_AR_THRESH:
                self.yawn_counter += 1
                if self.yawn_counter >= self.YAWN_CONSEC_FRAMES:
                    if not self.yawn_alert:
                        self.yawn_alert = True
                        self.play_alert()
            else:
                self.yawn_counter = 0
                self.yawn_alert = False
            
            # Display metrics
            cv2.putText(frame, f"Driver: {self.verified_driver_name}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"EAR: {ear:.2f}", (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (10, 90),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Alert messages
            if self.drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 130),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            
            if self.yawn_alert:
                cv2.putText(frame, "YAWNING DETECTED!", (10, 170),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 3)
    
    def run_system(self):
        """Main system loop"""
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        print("=== INTEGRATED DRIVER SAFETY SYSTEM ===")
        print("Phase 1: Security Verification")
        print("Phase 2: Drowsiness Monitoring")
        print("Press 'q' to quit at any time")
        print(f"Alert sound: {self.mp3_file}")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip frame for mirror effect
            frame = cv2.flip(frame, 1)
            
            if not self.security_verified:
                # Security verification phase
                verification_complete = self.security_verification(frame)
                
                if verification_complete:
                    # Show success message
                    cv2.rectangle(frame, (50, 200), (600, 300), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, "ACCESS GRANTED!", (80, 230), 
                               cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 3)
                    cv2.putText(frame, f"Welcome {self.verified_driver_name}", (80, 270), 
                               cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    cv2.imshow('Driver Safety System', frame)
                    cv2.waitKey(2000)  # Show for 2 seconds
                    
                    print(f"\n=== SECURITY VERIFIED: {self.verified_driver_name} ===")
                    print("Switching to drowsiness monitoring...")
                    time.sleep(1)
            else:
                # Drowsiness monitoring phase
                self.drowsiness_monitoring(frame)
            
            # Common display elements
            cv2.putText(frame, "Press 'q' to quit", (10, frame.shape[0] - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # System status
            if self.security_verified:
                cv2.putText(frame, "STATUS: MONITORING", (10, frame.shape[0] - 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "STATUS: VERIFYING", (10, frame.shape[0] - 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow('Driver Safety System', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        if self.alert_sound:
            pygame.mixer.quit()
        
        print("\nDriver Safety System terminated.")

def main():
    """Main function to run the integrated system"""
    try:
        system = IntegratedDriverSafetySystem()
        system.run_system()
    except Exception as e:
        print(f"System error: {e}")
        print("Please check all requirements are installed and files are in place")

if __name__ == "__main__":
    main()

Found driver images: ['Ahmed.jpg', 'Bill.jpeg', 'Elon.jpeg', 'Mohamed Ali.jpg']
Authorized drivers: ['Ahmed', 'Bill', 'Elon', 'Mohamed Ali']
Successfully encoded 4 driver faces
Drowsiness detection system initialized
Audio system ready - Using: Untitled (1).mp3
=== INTEGRATED DRIVER SAFETY SYSTEM ===
Phase 1: Security Verification
Phase 2: Drowsiness Monitoring
Press 'q' to quit at any time
Alert sound: Untitled (1).mp3

=== SECURITY VERIFIED: AHMED ===
Switching to drowsiness monitoring...

Driver Safety System terminated.


In [1]:
# -*- coding: utf-8 -*-
"""
Integrated Driver Security and Drowsiness Detection System

This system combines:
1. Face recognition security verification (runs first)
2. Driver drowsiness and yawning detection (runs after security verification)

Requirements:
1. Install packages: pip install opencv-python numpy scipy pygame mediapipe
2. Create 'Images_sec' folder with authorized driver photos (e.g., Ahmed.jpg, Bill.jpeg, etc.)
3. Place your MP3 alert file "Untitled (1).mp3" in the same directory
4. Ensure webcam is connected and working

Usage:
- Run the script and position your face clearly in front of the camera
- First, the security system will verify your identity (needs 2 consecutive verifications)
- After successful verification, drowsiness monitoring begins automatically
- The system will play "Untitled (1).mp3" when drowsiness or yawning is detected
- Press 'q' to quit at any time
"""

import cv2
import numpy as np
from scipy.spatial import distance as dist
import pygame
import time
import sys
import os
import datetime
from collections import deque
import mediapipe as mp

class IntegratedDriverSafetySystem:
    def __init__(self, security_images_path="Images_sec", mp3_filename="Untitled (1).mp3"):
        # Security system initialization
        self.security_images_path = security_images_path
        self.authorized_drivers = [] # This will store face encodings if we re-implement face recognition
        self.driver_names = []
        # For simplicity, removing face recognition for now to focus on drowsiness
        # self.load_authorized_drivers() 
        
        # Drowsiness detection initialization
        self.face_mesh = None
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mp_face_mesh = mp.solutions.face_mesh
        self.init_drowsiness_detector()
        
        # System state
        self.security_verified = True # Auto-verify security for now
        self.verified_driver_name = "Driver"
        self.consecutive_verifications = 0
        self.last_verified_name = ""
        
        # Audio setup
        self.mp3_file = mp3_filename
        self.setup_audio()
        
        # Drowsiness detection parameters (MediaPipe indices)
        # These indices are for the 468 landmarks provided by MediaPipe Face Mesh
        # Left eye landmarks (approximate, based on common EAR implementations)
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE = [133, 173, 157, 158, 159, 160, 161, 246, 7, 33, 163, 144, 145, 153, 154, 155]
        # Mouth landmarks (approximate, for MAR calculation)
        self.MOUTH = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 309]

        # Thresholds (initial values, will be dynamically adjusted)
        self.EYE_AR_THRESH = 0.25
        self.MOUTH_AR_THRESH = 0.7
        self.EYE_AR_CONSEC_FRAMES = 20
        self.YAWN_CONSEC_FRAMES = 15
        
        # Counters for drowsiness detection
        self.eye_counter = 0
        self.yawn_counter = 0
        self.drowsy_alert = False
        self.yawn_alert = False

        # New attributes for dynamic thresholding and temporal analysis
        self.baseline_ear = None
        self.baseline_mar = None
        self.ear_history = deque(maxlen=60) # Store last 60 EAR values (approx 2 seconds at 30 FPS)
        self.blink_duration_frames = 0
        self.blink_count = 0
        self.last_blink_time = time.time()
        self.calibration_complete = False
        self.calibration_frames_collected = 0
        self.CALIBRATION_DURATION_SECONDS = 5 # Calibrate for 5 seconds
        self.CALIBRATION_EAR_SAMPLES = []
        self.CALIBRATION_MAR_SAMPLES = []

    def load_authorized_drivers(self):
        """Load and encode authorized driver faces"""
        # This method is currently not used as face recognition is temporarily removed.
        pass
    
    def init_drowsiness_detector(self):
        """Initialize MediaPipe Face Mesh for drowsiness detection"""
        try:
            self.face_mesh = self.mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
            print("Drowsiness detection system initialized with MediaPipe Face Mesh")
        except Exception as e:
            print(f"Error initializing drowsiness detector with MediaPipe: {e}")
            print("Make sure MediaPipe is installed and working correctly.")
    
    def setup_audio(self):
        """Setup audio system for alerts"""
        try:
            pygame.mixer.init()
            self.alert_sound = pygame.mixer.Sound(self.mp3_file)
            print(f"Audio system ready - Using: {self.mp3_file}")
        except Exception as e:
            print(f"Audio setup failed: {e}")
            print("Will use system beep as fallback")
            self.alert_sound = None
    
    def play_alert(self):
        """Play alert sound"""
        if self.alert_sound:
            self.alert_sound.play()
        else:
            print('\a')  # System beep
            sys.stdout.flush()
    
    def security_verification(self, frame):
        """Handle security verification phase"""
        # This method is currently bypassed for simplicity to focus on drowsiness detection.
        # In a real application, this would involve face detection and recognition.
        cv2.putText(frame, "SECURITY VERIFICATION BYPASSED", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, "Drowsiness monitoring starting...", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        return True
    
    def eye_aspect_ratio(self, eye):
        """Calculate eye aspect ratio"""
        # Compute the Euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        # Compute the Euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])
        # Compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)
        return ear
    
    def mouth_aspect_ratio(self, mouth):
        """Calculate mouth aspect ratio"""
        # Compute the Euclidean distances between the two sets of
        # vertical mouth landmarks (x, y)-coordinates
        A = dist.euclidean(mouth[2], mouth[10])
        B = dist.euclidean(mouth[4], mouth[8])
        # Compute the Euclidean distance between the horizontal
        # mouth landmark (x, y)-coordinates
        C = dist.euclidean(mouth[0], mouth[6])
        # Compute the mouth aspect ratio
        mar = (A + B) / (2.0 * C)
        return mar
    
    def extract_landmarks(self, frame):
        """Extract facial landmarks using MediaPipe Face Mesh"""
        h, w, c = frame.shape
        results = self.face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        landmarks = []
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for i in range(0, 468):
                    pt1 = face_landmarks.landmark[i]
                    x, y = int(pt1.x * w), int(pt1.y * h)
                    landmarks.append([x, y])
            return np.array(landmarks)
        return None

    def calibrate_thresholds(self, frame):
        """Calibrate EAR and MAR thresholds based on initial frames."""
        landmarks = self.extract_landmarks(frame)

        # Ensure all required landmarks are present before attempting to access them
        if landmarks is not None and len(landmarks) > max(max(self.LEFT_EYE), max(self.RIGHT_EYE), max(self.MOUTH)):
            left_eye = np.array([landmarks[i] for i in self.LEFT_EYE])
            right_eye = np.array([landmarks[i] for i in self.RIGHT_EYE])
            mouth = np.array([landmarks[i] for i in self.MOUTH])
            
            ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
            mar = self.mouth_aspect_ratio(mouth)

            self.CALIBRATION_EAR_SAMPLES.append(ear)
            self.CALIBRATION_MAR_SAMPLES.append(mar)
            self.calibration_frames_collected += 1

            cv2.putText(frame, "CALIBRATING... Please look at camera", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, f"Frames: {self.calibration_frames_collected}", (50, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            if self.calibration_frames_collected >= self.CALIBRATION_DURATION_SECONDS * 30: # Assuming 30 FPS
                self.baseline_ear = np.mean(self.CALIBRATION_EAR_SAMPLES)
                self.baseline_mar = np.mean(self.CALIBRATION_MAR_SAMPLES)
                
                # Set dynamic thresholds based on baseline. These values can be fine-tuned.
                self.EYE_AR_THRESH = self.baseline_ear * 0.75 # 25% drop from baseline
                self.MOUTH_AR_THRESH = self.baseline_mar * 1.25 # 25% increase from baseline
                
                self.calibration_complete = True
                print(f"Calibration complete. Baseline EAR: {self.baseline_ear:.2f}, New EAR Thresh: {self.EYE_AR_THRESH:.2f}")
                print(f"Baseline MAR: {self.baseline_mar:.2f}, New MAR Thresh: {self.MOUTH_AR_THRESH:.2f}")
        else:
            cv2.putText(frame, "No face detected for calibration!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        return self.calibration_complete

    def drowsiness_monitoring(self, frame):
        """Handle drowsiness monitoring phase"""
        landmarks = self.extract_landmarks(frame)
        
        if landmarks is not None and len(landmarks) > max(max(self.LEFT_EYE), max(self.RIGHT_EYE), max(self.MOUTH)):
            # Extract eye and mouth coordinates
            left_eye = np.array([landmarks[i] for i in self.LEFT_EYE])
            right_eye = np.array([landmarks[i] for i in self.RIGHT_EYE])
            mouth = np.array([landmarks[i] for i in self.MOUTH])
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = self.mouth_aspect_ratio(mouth)

            self.ear_history.append(ear)
            
            # Draw contours
            # MediaPipe drawing utilities can be used for more sophisticated drawing
            # For simplicity, we'll draw convex hulls using OpenCV for now
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)
            
            # Drowsiness detection (EAR)
            if ear < self.EYE_AR_THRESH:
                self.eye_counter += 1
                self.blink_duration_frames += 1
                if self.eye_counter >= self.EYE_AR_CONSEC_FRAMES:
                    if not self.drowsy_alert:
                        self.drowsy_alert = True
                        self.play_alert()
            else:
                if self.eye_counter > 0: # Eye just opened after being closed
                    # Check for blink duration (a normal blink is usually 3-5 frames)
                    if self.blink_duration_frames > 2 and self.blink_duration_frames < 15: # Avoid very short noise and very long closures
                        self.blink_count += 1
                        self.last_blink_time = time.time()
                self.eye_counter = 0
                self.blink_duration_frames = 0
                self.drowsy_alert = False
            
            # Yawning detection
            if mar > self.MOUTH_AR_THRESH:
                self.yawn_counter += 1
                if self.yawn_counter >= self.YAWN_CONSEC_FRAMES:
                    if not self.yawn_alert:
                        self.yawn_alert = True
                        self.play_alert()
            else:
                self.yawn_counter = 0
                self.yawn_alert = False
            
            # Calculate PERCLOS (Percentage of Eyelid Closure Over the Pupil Over Time)
            perclos = 0.0
            if len(self.ear_history) > 0:
                closed_frames = sum(1 for e in self.ear_history if e < self.EYE_AR_THRESH)
                perclos = (closed_frames / len(self.ear_history)) * 100

            # Display metrics
            cv2.putText(frame, f"Driver: {self.verified_driver_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"EAR: {ear:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"PERCLOS: {perclos:.1f}%", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Blinks: {self.blink_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Alert messages
            if self.drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            
            if self.yawn_alert:
                cv2.putText(frame, "YAWNING DETECTED!", (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 3)
        else:
            cv2.putText(frame, "No face detected for drowsiness monitoring!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    def run_system(self, video_path=None, output_video_path=None):
        """Main system loop"""
        if video_path:
            cap = cv2.VideoCapture(video_path)
        else:
            cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("Error: Could not open video stream or file.")
            return

        # Initialize VideoWriter only if output_video_path is provided
        out = None
        if output_video_path:
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        print("\n=== INTEGRATED DRIVER SAFETY SYSTEM ===")
        print("Phase 1: Security Verification (Bypassed)")
        print("Phase 2: Drowsiness Monitoring")
        if video_path:
            print(f"Processing video: {video_path}")
        else:
            print("Using webcam for live feed.")
        if output_video_path:
            print(f"Output video will be saved to: {output_video_path}")
        print(f"Alert sound: {self.mp3_file}")

        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video stream or file.")
                break

            frame = cv2.flip(frame, 1) # Mirror image

            if not self.security_verified:
                # This block is currently bypassed.
                self.security_verification(frame)
            elif not self.calibration_complete:
                self.calibrate_thresholds(frame)
            else:
                self.drowsiness_monitoring(frame)

            cv2.imshow("Driver Safety System", frame)

            # Write the processed frame to the output video file if writer is initialized
            if out:
                out.write(frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cap.release()
        if out:
            out.release()
        cv2.destroyAllWindows()
        print("\nDriver Safety System terminated.")

if __name__ == "__main__":
    # To use webcam, call run_system() without arguments or with video_path=None
    # To process a video file, provide the path: system.run_system("path/to/your/video.mp4")
    # To save output to a video fiqle, provide output_video_path: system.run_system(output_video_path="output.mp4")
    system = IntegratedDriverSafetySystem()
    system.run_system()




pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html

Drowsiness detection system initialized with MediaPipe Face Mesh
Audio system ready - Using: Untitled (1).mp3

=== INTEGRATED DRIVER SAFETY SYSTEM ===
Phase 1: Security Verification (Bypassed)
Phase 2: Drowsiness Monitoring
Using webcam for live feed.
Alert sound: Untitled (1).mp3
Calibration complete. Baseline EAR: 1.37, New EAR Thresh: 1.03
Baseline MAR: 1.14, New MAR Thresh: 1.43

Driver Safety System terminated.


## 2nd Mediapipe

In [1]:
# -*- coding: utf-8 -*-
"""
Integrated Driver Security and Drowsiness Detection System

This system combines:
1. Face recognition security verification (runs first)
2. Driver drowsiness and yawning detection (runs after security verification)

Requirements:
1. Install packages: pip install opencv-python numpy scipy pygame mediapipe face_recognition
2. Create 'Images_sec' folder with authorized driver photos (e.g., Ahmed.jpg, Bill.jpeg, etc.)
3. Place your MP3 alert file "Untitled (1).mp3" in the same directory
4. Ensure webcam is connected and working

Usage:
- Run the script and position your face clearly in front of the camera
- First, the security system will verify your identity (needs 2 consecutive verifications)
- After successful verification, drowsiness monitoring begins automatically
- The system will play "Untitled (1).mp3" when drowsiness or yawning is detected
- Press 'q' to quit at any time
"""

import cv2
import numpy as np
from scipy.spatial import distance as dist
import pygame
import time
import sys
import os
import datetime
from collections import deque
import mediapipe as mp
import face_recognition # Import face_recognition library

class IntegratedDriverSafetySystem:
    def __init__(self, security_images_path="Images_sec", mp3_filename="Untitled (1).mp3"):
        # Security system initialization
        self.security_images_path = security_images_path
        self.authorized_drivers = [] # This will store face encodings
        self.driver_names = []
        self.load_authorized_drivers() # Load authorized drivers
        
        # Drowsiness detection initialization
        self.face_mesh = None
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mp_face_mesh = mp.solutions.face_mesh
        self.init_drowsiness_detector()
        
        # System state
        self.security_verified = False # Set to False initially
        self.verified_driver_name = "Unknown"
        self.consecutive_verifications = 0
        self.last_verified_name = ""
        
        # Audio setup
        self.mp3_file = mp3_filename
        self.setup_audio()
        
        # Drowsiness detection parameters (MediaPipe indices)
        # These indices are for the 468 landmarks provided by MediaPipe Face Mesh
        # Left eye landmarks (approximate, based on common EAR implementations)
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE = [133, 173, 157, 158, 159, 160, 161, 246, 7, 33, 163, 144, 145, 153, 154, 155]
        # Mouth landmarks (approximate, for MAR calculation)
        self.MOUTH = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 309]

        # Thresholds (initial values, will be dynamically adjusted)
        self.EYE_AR_THRESH = 0.25
        self.MOUTH_AR_THRESH = 0.7
        self.EYE_AR_CONSEC_FRAMES = 20
        self.YAWN_CONSEC_FRAMES = 15
        
        # Counters for drowsiness detection
        self.eye_counter = 0
        self.yawn_counter = 0
        self.drowsy_alert = False
        self.yawn_alert = False

        # New attributes for dynamic thresholding and temporal analysis
        self.baseline_ear = None
        self.baseline_mar = None
        self.ear_history = deque(maxlen=60) # Store last 60 EAR values (approx 2 seconds at 30 FPS)
        self.blink_duration_frames = 0
        self.blink_count = 0
        self.last_blink_time = time.time()
        self.calibration_complete = False
        self.calibration_frames_collected = 0
        self.CALIBRATION_DURATION_SECONDS = 5 # Calibrate for 5 seconds
        self.CALIBRATION_EAR_SAMPLES = []
        self.CALIBRATION_MAR_SAMPLES = []

    def load_authorized_drivers(self):
        """Load and encode authorized driver faces"""
        print(f"Loading authorized drivers from {self.security_images_path}...")
        if not os.path.exists(self.security_images_path):
            print(f"Warning: Security images path '{self.security_images_path}' does not exist. Security verification will not function.")
            return

        for filename in os.listdir(self.security_images_path):
            if filename.endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(self.security_images_path, filename)
                image = face_recognition.load_image_file(image_path)
                face_encodings = face_recognition.face_encodings(image)
                
                if len(face_encodings) > 0:
                    self.authorized_drivers.append(face_encodings[0])
                    self.driver_names.append(os.path.splitext(filename)[0]) # Use filename as name
                    print(f"Loaded: {os.path.splitext(filename)[0]}")
                else:
                    print(f"No face found in {filename}")
        if not self.authorized_drivers:
            print("No authorized drivers loaded. Security verification will be skipped.")
    
    def init_drowsiness_detector(self):
        """Initialize MediaPipe Face Mesh for drowsiness detection"""
        try:
            self.face_mesh = self.mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
            print("Drowsiness detection system initialized with MediaPipe Face Mesh")
        except Exception as e:
            print(f"Error initializing drowsiness detector with MediaPipe: {e}")
            print("Make sure MediaPipe is installed and working correctly.")
    
    def setup_audio(self):
        """Setup audio system for alerts"""
        try:
            pygame.mixer.init()
            self.alert_sound = pygame.mixer.Sound(self.mp3_file)
            print(f"Audio system ready - Using: {self.mp3_file}")
        except Exception as e:
            print(f"Audio setup failed: {e}")
            print("Will use system beep as fallback")
            self.alert_sound = None
    
    def play_alert(self):
        """Play alert sound"""
        if self.alert_sound:
            self.alert_sound.play()
        else:
            print('\a')  # System beep
            sys.stdout.flush()
    
    def security_verification(self, frame):
        """Handle security verification phase"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        verified_current_frame = False
        current_frame_name = "Unknown"

        if not self.authorized_drivers:
            cv2.putText(frame, "NO AUTHORIZED DRIVERS LOADED!", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2)
            cv2.putText(frame, "Add images to Images_sec folder.", (50, 80), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 0, 255), 2)
            return False

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(self.authorized_drivers, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(self.authorized_drivers, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.driver_names[best_match_index]
                current_frame_name = name
                verified_current_frame = True

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0) if verified_current_frame else (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0) if verified_current_frame else (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.7, (255, 255, 255), 1)

        if verified_current_frame and current_frame_name == self.last_verified_name:
            self.consecutive_verifications += 1
            if self.consecutive_verifications >= 2: # Needs 2 consecutive verifications
                self.security_verified = True
                self.verified_driver_name = current_frame_name
                cv2.putText(frame, f"VERIFIED: {self.verified_driver_name}", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, "Drowsiness monitoring starting...", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            else:
                cv2.putText(frame, f"Verifying: {current_frame_name} ({self.consecutive_verifications}/2)", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        else:
            self.consecutive_verifications = 0
            self.last_verified_name = current_frame_name
            cv2.putText(frame, "UNAUTHORIZED!", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "Please verify your identity.", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        return self.security_verified
    
    def eye_aspect_ratio(self, eye):
        """Calculate eye aspect ratio"""
        # Compute the Euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        # Compute the Euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])
        # Compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)
        return ear
    
    def mouth_aspect_ratio(self, mouth):
        """Calculate mouth aspect ratio"""
        # Compute the Euclidean distances between the two sets of
        # vertical mouth landmarks (x, y)-coordinates
        A = dist.euclidean(mouth[2], mouth[10])
        B = dist.euclidean(mouth[4], mouth[8])
        # Compute the Euclidean distance between the horizontal
        # mouth landmark (x, y)-coordinates
        C = dist.euclidean(mouth[0], mouth[6])
        # Compute the mouth aspect ratio
        mar = (A + B) / (2.0 * C)
        return mar
    
    def extract_landmarks(self, frame):
        """Extract facial landmarks using MediaPipe Face Mesh"""
        h, w, c = frame.shape
        results = self.face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        landmarks = []
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for i in range(0, 468):
                    pt1 = face_landmarks.landmark[i]
                    x, y = int(pt1.x * w), int(pt1.y * h)
                    landmarks.append([x, y])
            return np.array(landmarks)
        return None

    def calibrate_thresholds(self, frame):
        """Calibrate EAR and MAR thresholds based on initial frames."""
        landmarks = self.extract_landmarks(frame)

        # Ensure all required landmarks are present before attempting to access them
        if landmarks is not None and len(landmarks) > max(max(self.LEFT_EYE), max(self.RIGHT_EYE), max(self.MOUTH)):
            left_eye = np.array([landmarks[i] for i in self.LEFT_EYE])
            right_eye = np.array([landmarks[i] for i in self.RIGHT_EYE])
            mouth = np.array([landmarks[i] for i in self.MOUTH])
            
            ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
            mar = self.mouth_aspect_ratio(mouth)

            self.CALIBRATION_EAR_SAMPLES.append(ear)
            self.CALIBRATION_MAR_SAMPLES.append(mar)
            self.calibration_frames_collected += 1

            cv2.putText(frame, "CALIBRATING... Please look at camera", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, f"Frames: {self.calibration_frames_collected}", (50, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            if self.calibration_frames_collected >= self.CALIBRATION_DURATION_SECONDS * 30: # Assuming 30 FPS
                self.baseline_ear = np.mean(self.CALIBRATION_EAR_SAMPLES)
                self.baseline_mar = np.mean(self.CALIBRATION_MAR_SAMPLES)
                
                # Set dynamic thresholds based on baseline. These values can be fine-tuned.
                self.EYE_AR_THRESH = self.baseline_ear * 0.75 # 25% drop from baseline
                self.MOUTH_AR_THRESH = self.baseline_mar * 1.25 # 25% increase from baseline
                
                self.calibration_complete = True
                print(f"Calibration complete. Baseline EAR: {self.baseline_ear:.2f}, New EAR Thresh: {self.EYE_AR_THRESH:.2f}")
                print(f"Baseline MAR: {self.baseline_mar:.2f}, New MAR Thresh: {self.MOUTH_AR_THRESH:.2f}")
        else:
            cv2.putText(frame, "No face detected for calibration!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        return self.calibration_complete

    def drowsiness_monitoring(self, frame):
        """Handle drowsiness monitoring phase"""
        landmarks = self.extract_landmarks(frame)
        
        if landmarks is not None and len(landmarks) > max(max(self.LEFT_EYE), max(self.RIGHT_EYE), max(self.MOUTH)):
            # Extract eye and mouth coordinates
            left_eye = np.array([landmarks[i] for i in self.LEFT_EYE])
            right_eye = np.array([landmarks[i] for i in self.RIGHT_EYE])
            mouth = np.array([landmarks[i] for i in self.MOUTH])
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = self.mouth_aspect_ratio(mouth)

            self.ear_history.append(ear)
            
            # Draw contours
            # MediaPipe drawing utilities can be used for more sophisticated drawing
            # For simplicity, we'll draw convex hulls using OpenCV for now
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)
            
            # Drowsiness detection (EAR)
            if ear < self.EYE_AR_THRESH:
                self.eye_counter += 1
                self.blink_duration_frames += 1
                if self.eye_counter >= self.EYE_AR_CONSEC_FRAMES:
                    if not self.drowsy_alert:
                        self.drowsy_alert = True
                        self.play_alert()
            else:
                if self.eye_counter > 0: # Eye just opened after being closed
                    # Check for blink duration (a normal blink is usually 3-5 frames)
                    if self.blink_duration_frames > 2 and self.blink_duration_frames < 15: # Avoid very short noise and very long closures
                        self.blink_count += 1
                        self.last_blink_time = time.time()
                self.eye_counter = 0
                self.blink_duration_frames = 0
                self.drowsy_alert = False
            
            # Yawning detection
            if mar > self.MOUTH_AR_THRESH:
                self.yawn_counter += 1
                if self.yawn_counter >= self.YAWN_CONSEC_FRAMES:
                    if not self.yawn_alert:
                        self.yawn_alert = True
                        self.play_alert()
            else:
                self.yawn_counter = 0
                self.yawn_alert = False
            
            # Calculate PERCLOS (Percentage of Eyelid Closure Over the Pupil Over Time)
            perclos = 0.0
            if len(self.ear_history) > 0:
                closed_frames = sum(1 for e in self.ear_history if e < self.EYE_AR_THRESH)
                perclos = (closed_frames / len(self.ear_history)) * 100

            # Display metrics
            cv2.putText(frame, f"Driver: {self.verified_driver_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"EAR: {ear:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"PERCLOS: {perclos:.1f}%", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Blinks: {self.blink_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Alert messages
            if self.drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            
            if self.yawn_alert:
                cv2.putText(frame, "YAWNING DETECTED!", (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 3)
        else:
            cv2.putText(frame, "No face detected for drowsiness monitoring!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    def run_system(self, video_path=None, output_video_path=None):
        """Main system loop"""
        if video_path:
            cap = cv2.VideoCapture(video_path)
        else:
            cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("Error: Could not open video stream or file.")
            return

        # Initialize VideoWriter only if output_video_path is provided
        out = None
        if output_video_path:
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        print("\n=== INTEGRATED DRIVER SAFETY SYSTEM ===")
        print("Phase 1: Security Verification")
        print("Phase 2: Drowsiness Monitoring")
        if video_path:
            print(f"Processing video: {video_path}")
        else:
            print("Using webcam for live feed.")
        if output_video_path:
            print(f"Output video will be saved to: {output_video_path}")
        print(f"Alert sound: {self.mp3_file}")

        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video stream or file.")
                break

            frame = cv2.flip(frame, 1) # Mirror image

            if not self.security_verified:
                self.security_verification(frame)
            elif not self.calibration_complete:
                self.calibrate_thresholds(frame)
            else:
                self.drowsiness_monitoring(frame)

            cv2.imshow("Driver Safety System", frame)

            # Write the processed frame to the output video file if writer is initialized
            if out:
                out.write(frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cap.release()
        if out:
            out.release()
        cv2.destroyAllWindows()
        print("\nDriver Safety System terminated.")

if __name__ == "__main__":
    # To use webcam, call run_system() without arguments or with video_path=None
    # To process a video file, provide the path: system.run_system("path/to/your/video.mp4")
    # To save output to a video file, provide output_video_path: system.run_system(output_video_path="output.mp4")
    system = IntegratedDriverSafetySystem()
    system.run_system()




pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html

Loading authorized drivers from Images_sec...
Loaded: Ahmed
Loaded: Bill
Loaded: Elon
Loaded: Mohamed Ali
Drowsiness detection system initialized with MediaPipe Face Mesh
Audio system ready - Using: Untitled (1).mp3

=== INTEGRATED DRIVER SAFETY SYSTEM ===
Phase 1: Security Verification
Phase 2: Drowsiness Monitoring
Using webcam for live feed.
Alert sound: Untitled (1).mp3
Calibration complete. Baseline EAR: 1.38, New EAR Thresh: 1.03
Baseline MAR: 1.19, New MAR Thresh: 1.48

Driver Safety System terminated.


## 3rd Mediapipe

In [4]:
# -*- coding: utf-8 -*-
"""
Integrated Driver Security and Drowsiness Detection System

This system combines:
1. Face recognition security verification (runs first)
2. Driver drowsiness and yawning detection (runs after security verification)

Requirements:
1. Install packages: pip install opencv-python numpy scipy pygame mediapipe face_recognition
2. Create 'Images_sec' folder with authorized driver photos (e.g., Ahmed.jpg, Bill.jpeg, etc.)
3. Place your MP3 alert file "Untitled (1).mp3" in the same directory
4. Ensure webcam is connected and working

Usage:
- Run the script and position your face clearly in front of the camera
- First, the security system will verify your identity (needs 2 consecutive verifications)
- After successful verification, drowsiness monitoring begins automatically
- The system will play "Untitled (1).mp3" when drowsiness or yawning is detected
- Press 'q' to quit at any time
"""

import cv2
import numpy as np
from scipy.spatial import distance as dist
import pygame
import time
import sys
import os
import datetime
from collections import deque
import mediapipe as mp
import face_recognition # Import face_recognition library

class IntegratedDriverSafetySystem:
    def __init__(self, security_images_path="Images_sec", mp3_filename="Untitled (1).mp3"):
        # Security system initialization
        self.security_images_path = security_images_path
        self.authorized_drivers = [] # This will store face encodings
        self.driver_names = []
        self.load_authorized_drivers() # Load authorized drivers
        
        # Drowsiness detection initialization
        self.face_mesh = None
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mp_face_mesh = mp.solutions.face_mesh
        self.init_drowsiness_detector()
        
        # System state
        self.security_verified = False # Set to False initially
        self.verified_driver_name = "Unknown"
        self.consecutive_verifications = 0
        self.last_verified_name = ""
        
        # Audio setup
        self.mp3_file = mp3_filename
        self.setup_audio()
        
        # Drowsiness detection parameters (MediaPipe indices)
        # These indices are for the 468 landmarks provided by MediaPipe Face Mesh
        # Left eye landmarks (approximate, based on common EAR implementations)
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE = [133, 173, 157, 158, 159, 160, 161, 246, 7, 33, 163, 144, 145, 153, 154, 155]
        
        # Mouth landmarks for MAR calculation (more standard 6 points for MAR)
        # These are approximate indices for the inner and outer mouth corners from MediaPipe's 468 landmarks
        # Using a common set of 6 points for MAR: 13, 14, 78, 81, 308, 311
        self.MOUTH = [13, 14, 78, 81, 308, 311] # Updated indices for MAR calculation

        # Thresholds (initial values, will be dynamically adjusted)
        self.EYE_AR_THRESH = 0.25
        self.MOUTH_AR_THRESH = 0.7 # Will be dynamically adjusted, this is a fallback
        self.EYE_AR_CONSEC_FRAMES = 20
        self.YAWN_CONSEC_FRAMES = 15
        
        # Counters for drowsiness detection
        self.eye_counter = 0
        self.yawn_counter = 0
        self.drowsy_alert = False
        self.yawn_alert = False

        # New attributes for dynamic thresholding and temporal analysis
        self.baseline_ear = None
        self.baseline_mar = None
        self.ear_history = deque(maxlen=60) # Store last 60 EAR values (approx 2 seconds at 30 FPS)
        self.blink_duration_frames = 0
        self.blink_count = 0
        self.last_blink_time = time.time()
        self.calibration_complete = False
        self.calibration_frames_collected = 0
        self.CALIBRATION_DURATION_SECONDS = 5 # Calibrate for 5 seconds
        self.CALIBRATION_EAR_SAMPLES = []
        self.CALIBRATION_MAR_SAMPLES = []

    def load_authorized_drivers(self):
        """Load and encode authorized driver faces"""
        print(f"Loading authorized drivers from {self.security_images_path}...")
        if not os.path.exists(self.security_images_path):
            print(f"Warning: Security images path '{self.security_images_path}' does not exist. Security verification will not function.")
            return

        for filename in os.listdir(self.security_images_path):
            if filename.endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(self.security_images_path, filename)
                image = face_recognition.load_image_file(image_path)
                face_encodings = face_recognition.face_encodings(image)
                
                if len(face_encodings) > 0:
                    self.authorized_drivers.append(face_encodings[0])
                    self.driver_names.append(os.path.splitext(filename)[0]) # Use filename as name
                    print(f"Loaded: {os.path.splitext(filename)[0]}")
                else:
                    print(f"No face found in {filename}")
        if not self.authorized_drivers:
            print("No authorized drivers loaded. Security verification will be skipped.")
    
    def init_drowsiness_detector(self):
        """Initialize MediaPipe Face Mesh for drowsiness detection"""
        try:
            self.face_mesh = self.mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
            print("Drowsiness detection system initialized with MediaPipe Face Mesh")
        except Exception as e:
            print(f"Error initializing drowsiness detector with MediaPipe: {e}")
            print("Make sure MediaPipe is installed and working correctly.")
    
    def setup_audio(self):
        """Setup audio system for alerts"""
        try:
            pygame.mixer.init()
            self.alert_sound = pygame.mixer.Sound(self.mp3_file)
            print(f"Audio system ready - Using: {self.mp3_file}")
        except Exception as e:
            print(f"Audio setup failed: {e}")
            print("Will use system beep as fallback")
            self.alert_sound = None
    
    def play_alert(self):
        """Play alert sound"""
        if self.alert_sound:
            self.alert_sound.play()
        else:
            print('\a')  # System beep
            sys.stdout.flush()
    
    def security_verification(self, frame):
        """Handle security verification phase"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        verified_current_frame = False
        current_frame_name = "Unknown"

        if not self.authorized_drivers:
            cv2.putText(frame, "NO AUTHORIZED DRIVERS LOADED!", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2)
            cv2.putText(frame, "Add images to Images_sec folder.", (50, 80), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 0, 255), 2)
            return False

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(self.authorized_drivers, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(self.authorized_drivers, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.driver_names[best_match_index]
                current_frame_name = name
                verified_current_frame = True

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0) if verified_current_frame else (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0) if verified_current_frame else (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.7, (255, 255, 255), 1)

        if verified_current_frame and current_frame_name == self.last_verified_name:
            self.consecutive_verifications += 1
            if self.consecutive_verifications >= 2: # Needs 2 consecutive verifications
                self.security_verified = True
                self.verified_driver_name = current_frame_name
                cv2.putText(frame, f"VERIFIED: {self.verified_driver_name}", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, "Drowsiness monitoring starting...", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            else:
                cv2.putText(frame, f"Verifying: {current_frame_name} ({self.consecutive_verifications}/2)", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        else:
            self.consecutive_verifications = 0
            self.last_verified_name = current_frame_name
            cv2.putText(frame, "UNAUTHORIZED!", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "Please verify your identity.", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    def eye_aspect_ratio(self, eye):
        """Calculate eye aspect ratio"""
        # Compute the Euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        # Compute the Euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])
        # Compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)
        return ear
    
    def mouth_aspect_ratio(self, mouth):
        """Calculate mouth aspect ratio (MAR)"""
        # Using 6 points: 2 vertical, 2 horizontal
        # A: 13 (upper inner lip) to 14 (lower inner lip)
        # B: 78 (upper outer lip) to 81 (lower outer lip)
        # C: 308 (left mouth corner) to 311 (right mouth corner)
        
        # Vertical distances
        A = dist.euclidean(mouth[1], mouth[5]) # 14 to 311 (vertical inner)
        B = dist.euclidean(mouth[2], mouth[4]) # 78 to 308 (vertical outer)
        
        # Horizontal distance
        C = dist.euclidean(mouth[0], mouth[3]) # 13 to 81 (horizontal)
        
        # Calculate MAR
        mar = (A + B) / (2.0 * C)
        return mar
    
    def extract_landmarks(self, frame):
        """Extract facial landmarks using MediaPipe Face Mesh"""
        h, w, c = frame.shape
        results = self.face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        landmarks = []
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for i in range(0, 468):
                    pt1 = face_landmarks.landmark[i]
                    x, y = int(pt1.x * w), int(pt1.y * h)
                    landmarks.append([x, y])
            return np.array(landmarks)
        return None

    def calibrate_thresholds(self, frame):
        """Calibrate EAR and MAR thresholds based on initial frames."""
        landmarks = self.extract_landmarks(frame)

        # Ensure all required landmarks are present before attempting to access them
        if landmarks is not None and len(landmarks) > max(max(self.LEFT_EYE), max(self.RIGHT_EYE), max(self.MOUTH)):
            left_eye = np.array([landmarks[i] for i in self.LEFT_EYE])
            right_eye = np.array([landmarks[i] for i in self.RIGHT_EYE])
            mouth = np.array([landmarks[i] for i in self.MOUTH])
            
            ear = (self.eye_aspect_ratio(left_eye) + self.eye_aspect_ratio(right_eye)) / 2.0
            mar = self.mouth_aspect_ratio(mouth)

            self.CALIBRATION_EAR_SAMPLES.append(ear)
            self.CALIBRATION_MAR_SAMPLES.append(mar)
            self.calibration_frames_collected += 1

            cv2.putText(frame, "CALIBRATING... Please look at camera", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, f"Frames: {self.calibration_frames_collected}", (50, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            if self.calibration_frames_collected >= self.CALIBRATION_DURATION_SECONDS * 30: # Assuming 30 FPS
                self.baseline_ear = np.mean(self.CALIBRATION_EAR_SAMPLES)
                self.baseline_mar = np.mean(self.CALIBRATION_MAR_SAMPLES)
                
                # Set dynamic thresholds based on baseline. These values can be fine-tuned.
                self.EYE_AR_THRESH = self.baseline_ear * 0.75 # 25% drop from baseline
                self.MOUTH_AR_THRESH = self.baseline_mar * 1.25 # 25% increase from baseline
                
                self.calibration_complete = True
                print(f"Calibration complete. Baseline EAR: {self.baseline_ear:.2f}, New EAR Thresh: {self.EYE_AR_THRESH:.2f}")
                print(f"Baseline MAR: {self.baseline_mar:.2f}, New MAR Thresh: {self.MOUTH_AR_THRESH:.2f}")
        else:
            cv2.putText(frame, "No face detected for calibration!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        return self.calibration_complete

    def drowsiness_monitoring(self, frame):
        """Handle drowsiness monitoring phase"""
        landmarks = self.extract_landmarks(frame)
        
        if landmarks is not None and len(landmarks) > max(max(self.LEFT_EYE), max(self.RIGHT_EYE), max(self.MOUTH)):
            # Extract eye and mouth coordinates
            left_eye = np.array([landmarks[i] for i in self.LEFT_EYE])
            right_eye = np.array([landmarks[i] for i in self.RIGHT_EYE])
            mouth = np.array([landmarks[i] for i in self.MOUTH])
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = self.mouth_aspect_ratio(mouth)

            self.ear_history.append(ear)
            
            # Draw contours
            # MediaPipe drawing utilities can be used for more sophisticated drawing
            # For simplicity, we'll draw convex hulls using OpenCV for now
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)
            
            # Drowsiness detection (EAR)
            if ear < self.EYE_AR_THRESH:
                self.eye_counter += 1
                self.blink_duration_frames += 1
                if self.eye_counter >= self.EYE_AR_CONSEC_FRAMES:
                    if not self.drowsy_alert:
                        self.drowsy_alert = True
                        self.play_alert()
            else:
                if self.eye_counter > 0: # Eye just opened after being closed
                    # Check for blink duration (a normal blink is usually 3-5 frames)
                    if self.blink_duration_frames > 2 and self.blink_duration_frames < 15: # Avoid very short noise and very long closures
                        self.blink_count += 1
                        self.last_blink_time = time.time()
                self.eye_counter = 0
                self.blink_duration_frames = 0
                self.drowsy_alert = False
            
            # Yawning detection
            if mar > self.MOUTH_AR_THRESH:
                self.yawn_counter += 1
                if self.yawn_counter >= self.YAWN_CONSEC_FRAMES:
                    if not self.yawn_alert:
                        self.yawn_alert = True
                        self.play_alert()
            else:
                self.yawn_counter = 0
                self.yawn_alert = False
            
            # Calculate PERCLOS (Percentage of Eyelid Closure Over the Pupil Over Time)
            perclos = 0.0
            if len(self.ear_history) > 0:
                closed_frames = sum(1 for e in self.ear_history if e < self.EYE_AR_THRESH)
                perclos = (closed_frames / len(self.ear_history)) * 100

            # Display metrics
            cv2.putText(frame, f"Driver: {self.verified_driver_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"EAR: {ear:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"PERCLOS: {perclos:.1f}%", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Blinks: {self.blink_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Alert messages
            if self.drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            
            if self.yawn_alert:
                cv2.putText(frame, "YAWNING DETECTED!", (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 3)
        else:
            cv2.putText(frame, "No face detected for drowsiness monitoring!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    def run_system(self, video_path=None, output_video_path=None):
        """Main system loop"""
        if video_path:
            cap = cv2.VideoCapture(video_path)
        else:
            cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("Error: Could not open video stream or file.")
            return

        # Initialize VideoWriter only if output_video_path is provided
        out = None
        if output_video_path:
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        print("\n=== INTEGRATED DRIVER SAFETY SYSTEM ===")
        print("Phase 1: Security Verification")
        print("Phase 2: Drowsiness Monitoring")
        if video_path:
            print(f"Processing video: {video_path}")
        else:
            print("Using webcam for live feed.")
        if output_video_path:
            print(f"Output video will be saved to: {output_video_path}")
        print(f"Alert sound: {self.mp3_file}")

        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video stream or file.")
                break

            frame = cv2.flip(frame, 1) # Mirror image

            if not self.security_verified:
                self.security_verification(frame)
            elif not self.calibration_complete:
                self.calibrate_thresholds(frame)
            else:
                self.drowsiness_monitoring(frame)

            cv2.imshow("Driver Safety System", frame)

            # Write the processed frame to the output video file if writer is initialized
            if out:
                out.write(frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cap.release()
        if out:
            out.release()
        cv2.destroyAllWindows()
        print("\nDriver Safety System terminated.")

if __name__ == "__main__":
    # To use webcam, call run_system() without arguments or with video_path=None
    # To process a video file, provide the path: system.run_system("path/to/your/video.mp4")
    # To save output to a video file, provide output_video_path: system.run_system(output_video_path="output.mp4")
    system = IntegratedDriverSafetySystem()
    system.run_system()




Loading authorized drivers from Images_sec...
Loaded: Ahmed
Loaded: Bill
Loaded: Elon
Loaded: Mohamed Ali
Drowsiness detection system initialized with MediaPipe Face Mesh
Audio system ready - Using: Untitled (1).mp3

=== INTEGRATED DRIVER SAFETY SYSTEM ===
Phase 1: Security Verification
Phase 2: Drowsiness Monitoring
Using webcam for live feed.
Alert sound: Untitled (1).mp3
Calibration complete. Baseline EAR: 1.33, New EAR Thresh: 1.00
Baseline MAR: 2.55, New MAR Thresh: 3.19

Driver Safety System terminated.
